In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve, roc_auc_score, confusion_matrix
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Load dataset
df = pd.read_excel('../data/processed/exploratory_analysis_final.xlsx', sheet_name='Sheet1')

# --- Feature Engineering ---
# Standardize GRE scores (already have gre_avg but adding more features)
df['gre_total'] = df['gre_quantitative_reasoning'] + df['gre_verbal_reasoning']

# Interaction features
df['gpa_gre_interaction'] = df['undergrad_gpa'] * df['gre_avg']
df['gpa_acceptance_interaction'] = df['undergrad_gpa'] * df['acceptance_rate']


In [5]:

# Composite profile strength - revised to better weight importance factors
df['profile_strength'] = (df['gre_avg'] * 0.4 + 
                          df['gpa_percentile'] * 0.4 + 
                          (1 - df['acceptance_rate']) * 0.2)

# University rank bucketing - improved with more granular tiers
def rank_bucket(score):
    if pd.isna(score):
        return 'Unknown'
    elif score <= -5:
        return 'Elite (Top 5)'
    elif score <= -10:
        return 'Top 10'
    elif score <= -25:
        return 'Top 25'
    elif score <= -50:
        return 'Top 50'
    elif score <= -100:
        return 'Top 100'
    else:
        return 'Other'

df['qs_rank_bucket'] = df['qs_rank_score'].apply(rank_bucket)

# Calculate GRE percentile (estimated) - new feature
def gre_percentile(score):
    # Rough estimate of GRE percentile based on total score
    if score >= 330:
        return 0.95
    elif score >= 325:
        return 0.90
    elif score >= 320:
        return 0.85
    elif score >= 315:
        return 0.75
    elif score >= 310:
        return 0.65
    elif score >= 305:
        return 0.55
    elif score >= 300:
        return 0.45
    else:
        return 0.35


In [6]:

df['gre_percentile'] = df['gre_total'].apply(gre_percentile)

# Combined competitiveness score - new feature
df['competitiveness'] = (df['gre_percentile'] + df['gpa_percentile']) / 2

# Features to use
features = [
    # Original features
    'undergrad_gpa', 'gre_quantitative_reasoning', 'gre_verbal_reasoning',
    'analytical_writing', 'acceptance_rate', 'qs_rank_score', 'qs_tier',
    'gpa_percentile', 'gre_avg',
    
    # Engineered features
    'gre_total', 'gre_percentile', 'competitiveness',
    'gpa_gre_interaction', 'gpa_acceptance_interaction',
    'profile_strength', 'qs_rank_bucket'
]

# Create binary target: 1 if admitted, 0 otherwise
df['binary_decision'] = df['decision_grouped'].apply(lambda x: 1 if x == 1 else 0)

# Check class distribution
class_counts = df['binary_decision'].value_counts()
print(f"Class distribution: {class_counts}")
print(f"Class 1 percentage: {class_counts[1] / len(df) * 100:.2f}%")

# Display correlation heatmap for key features
correlation_features = ['undergrad_gpa', 'gre_avg', 'gre_total', 'acceptance_rate', 
                       'profile_strength', 'competitiveness', 'binary_decision']
correlation = df[correlation_features].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.savefig('correlation_heatmap.png')
plt.close()


Class distribution: binary_decision
0    64635
1    35986
Name: count, dtype: int64
Class 1 percentage: 35.76%


In [7]:

# Split data with stratification to maintain class proportions
X = df[features]
y = df['binary_decision']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Calculate class weight
class_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
print(f"Class weight for minority class: {class_weight:.2f}")

# Identify numeric and categorical features
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Changed to median for robustness
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# --- APPROACH 1: XGBoost with class weights ---
xgb_weighted_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        random_state=42,
        eval_metric='logloss',
        scale_pos_weight=class_weight  # Apply class weight
    ))
])

# --- APPROACH 2: SMOTE oversampling ---
smote_pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42, sampling_strategy=0.7)),  # Not fully balanced to prevent overfitting
    ('classifier', XGBClassifier(random_state=42, eval_metric='logloss'))
])


Class weight for minority class: 1.80


In [8]:

# --- Grid search parameters ---
# Reduced parameter grid for efficiency
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.05, 0.1],
    'classifier__subsample': [0.8, 1.0],
    'classifier__min_child_weight': [1, 3],
    'classifier__gamma': [0, 0.1],
    'classifier__colsample_bytree': [0.8, 1.0]
}

# Stratified CV for robust evaluation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid search for approach 1 (weighted)
print("\nTraining weighted XGBoost model with class balancing...")
grid_search_weighted = GridSearchCV(
    xgb_weighted_pipeline, param_grid, cv=cv, 
    scoring='f1', verbose=2, n_jobs=-1
)
grid_search_weighted.fit(X_train, y_train)

# Grid search for approach 2 (SMOTE)
print("\nTraining SMOTE XGBoost model...")
grid_search_smote = GridSearchCV(
    smote_pipeline, param_grid, cv=cv, 
    scoring='f1', verbose=2, n_jobs=-1
)
grid_search_smote.fit(X_train, y_train)

# --- Model Evaluation ---
# Function to evaluate and compare models
def evaluate_model(model, name):
    # Predict probabilities
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # ROC AUC score
    roc_auc = roc_auc_score(y_test, y_prob)
    
    # Find optimal threshold
    precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)
    f1_scores = np.zeros_like(thresholds)
    for i, threshold in enumerate(thresholds):
        y_pred_i = (y_prob >= threshold).astype(int)
        precision = np.sum((y_pred_i == 1) & (y_test == 1)) / max(np.sum(y_pred_i == 1), 1)
        recall = np.sum((y_pred_i == 1) & (y_test == 1)) / max(np.sum(y_test == 1), 1)
        f1_scores[i] = 2 * (precision * recall) / max((precision + recall), 1e-10)
    
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx]
    
    # Predictions with optimal threshold
    y_pred_optimal = (y_prob >= optimal_threshold).astype(int)
    
    # Metrics
    accuracy = accuracy_score(y_test, y_pred_optimal)
    report = classification_report(y_test, y_pred_optimal)
    conf_matrix = confusion_matrix(y_test, y_pred_optimal)
    
    # Print results
    print(f"\n--- {name} Evaluation ---")
    print(f"Best Parameters: {model.best_params_}")
    print(f"Optimal Threshold: {optimal_threshold:.4f}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(report)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Rejected', 'Admitted'],
                yticklabels=['Rejected', 'Admitted'])
    plt.title(f'Confusion Matrix - {name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_{name}.png')
    plt.close()
    
    return model.best_estimator_, y_prob, optimal_threshold

# Evaluate both models
weighted_model, weighted_probs, weighted_threshold = evaluate_model(grid_search_weighted, "Weighted XGBoost")
smote_model, smote_probs, smote_threshold = evaluate_model(grid_search_smote, "SMOTE XGBoost")

# --- Feature Importance Analysis ---
def plot_feature_importance(model, model_name):
    # Extract feature names
    feature_names = (
        numeric_features +
        list(model.named_steps['preprocessor']
             .named_transformers_['cat']
             .named_steps['onehot']
             .get_feature_names_out(categorical_features))
    )
    
    # Get importances
    importances = model.named_steps['classifier'].feature_importances_
    indices = np.argsort(importances)[::-1]
    
    # Plot
    plt.figure(figsize=(12, 8))
    plt.title(f'Feature Importance - {model_name}')
    plt.barh(range(min(20, len(feature_names))), 
             importances[indices[:20]], 
             align='center')
    plt.yticks(range(min(20, len(feature_names))), [feature_names[i] for i in indices[:20]])
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.savefig(f'feature_importance_{model_name}.png')
    plt.close()
    
    # Print top 10 features
    print(f"\nTop 10 features for {model_name}:")
    for i in range(min(10, len(feature_names))):
        print(f"{i+1}. {feature_names[indices[i]]} ({importances[indices[i]]:.4f})")

# Analyze feature importance for both models
plot_feature_importance(weighted_model, "Weighted_XGBoost")
plot_feature_importance(smote_model, "SMOTE_XGBoost")

# --- Ensemble Prediction ---
print("\n--- Ensemble Model ---")
# Combine predictions from both models
ensemble_probs = (weighted_probs + smote_probs) / 2

# Find optimal threshold for ensemble
precisions, recalls, thresholds = precision_recall_curve(y_test, ensemble_probs)
f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / (precisions[:-1] + recalls[:-1])
optimal_idx = np.argmax(f1_scores)
ensemble_threshold = thresholds[optimal_idx]

# Evaluate ensemble model
ensemble_preds = (ensemble_probs >= ensemble_threshold).astype(int)
ensemble_accuracy = accuracy_score(y_test, ensemble_preds)
ensemble_report = classification_report(y_test, ensemble_preds)
ensemble_conf_matrix = confusion_matrix(y_test, ensemble_preds)
ensemble_roc_auc = roc_auc_score(y_test, ensemble_probs)

print(f"Ensemble Optimal Threshold: {ensemble_threshold:.4f}")
print(f"Ensemble ROC AUC Score: {ensemble_roc_auc:.4f}")
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")
print(ensemble_report)

# Visualize ROC curves
plt.figure(figsize=(10, 8))
from sklearn.metrics import roc_curve



Training weighted XGBoost model with class balancing...
Fitting 5 folds for each of 192 candidates, totalling 960 fits
[CV] END classifier__colsample_bytree=0.8, classifier__gamma=0, classifier__learning_rate=0.05, classifier__max_depth=3, classifier__min_child_weight=1, classifier__n_estimators=100, classifier__subsample=0.8; total time=   0.5s
[CV] END classifier__colsample_bytree=0.8, classifier__gamma=0, classifier__learning_rate=0.05, classifier__max_depth=3, classifier__min_child_weight=1, classifier__n_estimators=100, classifier__subsample=0.8; total time=   0.6s
[CV] END classifier__colsample_bytree=0.8, classifier__gamma=0, classifier__learning_rate=0.05, classifier__max_depth=3, classifier__min_child_weight=1, classifier__n_estimators=100, classifier__subsample=1.0; total time=   0.6s
[CV] END classifier__colsample_bytree=0.8, classifier__gamma=0, classifier__learning_rate=0.05, classifier__max_depth=3, classifier__min_child_weight=1, classifier__n_estimators=100, classifier

<Figure size 1000x800 with 0 Axes>

In [9]:

# Plot ROC for weighted model
fpr_w, tpr_w, _ = roc_curve(y_test, weighted_probs)
plt.plot(fpr_w, tpr_w, label=f'Weighted XGBoost (AUC = {roc_auc_score(y_test, weighted_probs):.4f})')

# Plot ROC for SMOTE model
fpr_s, tpr_s, _ = roc_curve(y_test, smote_probs)
plt.plot(fpr_s, tpr_s, label=f'SMOTE XGBoost (AUC = {roc_auc_score(y_test, smote_probs):.4f})')

# Plot ROC for ensemble
fpr_e, tpr_e, _ = roc_curve(y_test, ensemble_probs)
plt.plot(fpr_e, tpr_e, label=f'Ensemble (AUC = {ensemble_roc_auc:.4f})')

# Plot diagonal
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend()
plt.tight_layout()
plt.savefig('roc_curves_comparison.png')
plt.close()

# --- Save final model ---
import pickle

# Save the best performing model (based on results)
best_model = weighted_model if roc_auc_score(y_test, weighted_probs) > roc_auc_score(y_test, smote_probs) else smote_model
best_threshold = weighted_threshold if roc_auc_score(y_test, weighted_probs) > roc_auc_score(y_test, smote_probs) else smote_threshold

# Save model and threshold
with open('admission_prediction_model.pkl', 'wb') as f:
    pickle.dump((best_model, best_threshold), f)

print("\n--- Model Training Complete ---")
print(f"Best model saved: {'Weighted XGBoost' if best_model == weighted_model else 'SMOTE XGBoost'}")
print(f"Best model threshold: {best_threshold:.4f}")

# --- Create a sample prediction function ---
def predict_admission_chance(student_data, model=best_model, threshold=best_threshold):
    """
    Predict admission chance for a new student
    
    Parameters:
    student_data: dict containing student information with the same features as training data
    model: trained model
    threshold: optimal threshold for prediction
    
    Returns:
    probability: float - probability of admission
    decision: bool - admission decision
    """
    # Convert to DataFrame
    student_df = pd.DataFrame([student_data])
    
    # Feature engineering (must match training pipeline)
    student_df['gre_total'] = student_df['gre_quantitative_reasoning'] + student_df['gre_verbal_reasoning']
    student_df['gpa_gre_interaction'] = student_df['undergrad_gpa'] * student_df['gre_avg']
    student_df['gpa_acceptance_interaction'] = student_df['undergrad_gpa'] * student_df['acceptance_rate']
    student_df['profile_strength'] = (student_df['gre_avg'] * 0.4 + 
                                     student_df['gpa_percentile'] * 0.4 + 
                                     (1 - student_df['acceptance_rate']) * 0.2)
    student_df['qs_rank_bucket'] = student_df['qs_rank_score'].apply(rank_bucket)
    student_df['gre_percentile'] = student_df['gre_total'].apply(gre_percentile)
    student_df['competitiveness'] = (student_df['gre_percentile'] + student_df['gpa_percentile']) / 2
    
    # Predict
    prob = model.predict_proba(student_df[features])[0, 1]
    decision = prob >= threshold
    
    return prob, decision

# Example usage
print("\n--- Example Prediction ---")
sample_student = {
    'undergrad_gpa': 3.8,
    'gre_quantitative_reasoning': 167,
    'gre_verbal_reasoning': 160,
    'analytical_writing': 5.0,
    'acceptance_rate': 0.15,
    'qs_rank_score': -25,  # Top 25 school
    'qs_tier': 1,
    'gpa_percentile': 0.85,
    'gre_avg': 163.5
}

prob, decision = predict_admission_chance(sample_student)
print(f"Sample student admission probability: {prob:.4f}")
print(f"Admission decision: {'Admitted' if decision else 'Rejected'}")


--- Model Training Complete ---
Best model saved: Weighted XGBoost
Best model threshold: 0.3777

--- Example Prediction ---
Sample student admission probability: 0.3716
Admission decision: Rejected
